In [1]:
using Oceananigans
using JLD2
using CairoMakie
using Statistics
using ImageFiltering: imfilter, Kernel.gaussian
using ZipFile

[ Info: Oceananigans will use 4 threads


In [2]:
runname = "default"
@inline function makefluxvid(runname, i=nothing)
    foldername = "../scratch/filament-instability/$runname"
    filename = "down_front_mean.jld2"
    qRifilename = "qRi.jld2"
    fluxfilename = "turbulent_flux.jld2"
    paramfilename = "parameters.jld2"
    frames, grid = jldopen("$foldername/$filename") do file
        keys(file["timeseries/t"]), file["serialized/grid"]
        end;
    xᶜᵃᵃ = xnodes(Center, grid)
    xᶠᵃᵃ = xnodes(Face, grid)
    zᵃᵃᶜ = znodes(Center, grid)
    zᵃᵃᶠ = znodes(Face, grid)
    function ψᶜᶜᶜ(uᶠᶜᶜ, wᶜᶜᶠ, xᶜᵃᵃ, xᶠᵃᵃ, zᵃᵃᶜ, zᵃᵃᶠ)
        # Integrate
        Δzᵃᵃᶜ = reshape(diff(zᵃᵃᶠ), 1, length(zᵃᵃᶜ))
        Δx = xᶠᵃᵃ[2] - xᶠᵃᵃ[1]
        aᶠᶜᶜ = cumsum(uᶠᶜᶜ .* Δzᵃᵃᶜ; dims=3)

        bᶜᶜᶠ = cumsum(wᶜᶜᶠ .* Δx; dims=1)
        aᶜᶜᶜ = (circshift(aᶠᶜᶜ, (-1, 0)) .+ aᶠᶜᶜ) / 2
        bᶜᶜᶜ = (bᶜᶜᶠ[:, 1:end-1] .+ bᶜᶜᶠ[:, 2:end]) ./ 2
        return -aᶜᶜᶜ .+ bᶜᶜᶜ
    end

    file = jldopen("$foldername/$filename")
    qRifile = jldopen("$foldername/$qRifilename")
    fluxfile = jldopen("$foldername/$fluxfilename")
    
    n = Observable(101)

    frame = @lift frames[$n]


    ts = [file["timeseries/t/$f"] for f in frames] .- 30

    u = @lift file["timeseries/u_dfm/$($frame)"][:, 1, :]
    w = @lift file["timeseries/w_dfm/$($frame)"][:, 1, :]

    b = @lift file["timeseries/b_dfm/$($frame)"][:, 1, :]
    
    
    # Get the secondary cirulation streamfunction
    σ=3
    ψ = @lift imfilter(ψᶜᶜᶜ($u, $w, xᶜᵃᵃ, xᶠᵃᵃ, zᵃᵃᶜ, zᵃᵃᶠ), gaussian((σ, 0), (4σ+1, 5)), "circular")
    
    uflux = @lift fluxfile["timeseries/uFLUX/$($frame)"][:, 1, :]
    vflux = @lift fluxfile["timeseries/vFLUX/$($frame)"][:, 1, :]
    wflux = @lift fluxfile["timeseries/wFLUX/$($frame)"][:, 1, :]
    bflux = @lift fluxfile["timeseries/bFLUX/$($frame)"][:, 1, :]
    q = @lift imfilter(qRifile["timeseries/q/$($frame)"][:, 1, :], gaussian((σ, 0), (4σ+1, 5)), "circular")
    q_check = @lift ifelse.($q .< 0, 1, NaN)
    invRi = @lift imfilter(qRifile["timeseries/invRi/$($frame)"][:, 1, :], gaussian((σ, 0), (4σ+1, 5)), "circular")
    invRi_check = @lift ifelse.(1/0.95 .< $invRi .< 4, 1, NaN)
    title = @lift "t = $(round(ts[$n]; digits=2))"
    
    axis_kwargs = (; xlabel="x", ylabel="z", title, limits=(-2, 0, -0.12, 0))

    fig = Figure(resolution=(1200, 800))
    fig[1, 1:4] = Label(fig, title, fontsize=20)
    axu = Axis(fig[2, 1]; title=L"\langle w'u'\rangle", axis_kwargs...)
    axv = Axis(fig[2, 3]; title=L"\langle w'v'\rangle", axis_kwargs...)
    axw = Axis(fig[3, 1]; title=L"\langle w'w'\rangle", axis_kwargs...)
    axb = Axis(fig[3, 3]; title=L"\langle w'b'\rangle", axis_kwargs...)
    
    limuv = 1e-4
    limw = 1e-4
    limb = 1e-4
    htu = heatmap!(axu, xᶜᵃᵃ, zᵃᵃᶠ, uflux; colormap=:balance, colorrange=(-limuv, limuv))
    htv = heatmap!(axv, xᶜᵃᵃ, zᵃᵃᶠ, vflux; colormap=:balance, colorrange=(-limuv, limuv))
    htw = heatmap!(axw, xᶜᵃᵃ, zᵃᵃᶠ, wflux; colormap=:balance, colorrange=(-limw, limw))
    htb = heatmap!(axb, xᶜᵃᵃ, zᵃᵃᶠ, bflux; colormap=:balance, colorrange=(-limb, limb))
    
    for ax in [axu, axv, axw, axb]
        contour!(ax, xᶜᵃᵃ, zᵃᵃᶜ, ψ; colormap=:BrBG_10, levels=range(-0.012, 0.012, 40), alpha=1, linewidth=1)
        contour!(ax, xᶜᵃᵃ, zᵃᵃᶜ, b; color=(:black, 1), levels=range(-600, 200, 160), linewidth=1)
        #contour!(ax, xᶜᵃᵃ, zᵃᵃᶠ, invRi; color=:blue, levels=[1/0.95], alpha=0.5)
        #contour!(ax, xᶜᵃᵃ, zᵃᵃᶠ, invRi; color=:blue, levels=[1/0.25], linestyle=:dash, alpha=0.5)
        #contour!(ax, xᶜᵃᵃ, zᵃᵃᶜ, q; color=:red, levels=[0], alpha=0.5)
        #heatmap!(ax, xᶜᵃᵃ, zᵃᵃᶜ, q_check; colormap=[RGBAf(c.r, c.g, c.b, 0.1) for c in to_colormap(:reds)])
        #heatmap!(ax, xᶜᵃᵃ, zᵃᵃᶠ, invRi_check; colormap=[RGBAf(c.r, c.g, c.b, 0.1) for c in to_colormap(:blues)])
    end
    Colorbar(fig[2, 2], htu, label=L"\langle w'u'\rangle")
    Colorbar(fig[2, 4], htv, label=L"\langle w'v'\rangle")
    Colorbar(fig[3, 2], htw, label=L"\langle w'w'\rangle")
    Colorbar(fig[3, 4], htb, label=L"\langle w'b'\rangle")
    
    if i != nothing
        n[] = i
        return fig
    end
    
    vidfoldername = "output/videos/$runname/shearproduction"
    !ispath(vidfoldername) && mkpath(vidfoldername)
    w = ZipFile.Writer("$vidfoldername.zip");
    for i in 101:length(frames)
        n[] = i
        zipfile = ZipFile.addfile(w, "$(lpad(i, 4, '0')).png");
        save("$vidfoldername/$(lpad(i, 4, '0')).png", fig; resolution=(1200, 800))
        open(r -> write(zipfile, r), "$vidfoldername/$(lpad(i, 4, '0')).png")
        close(zipfile)
        rm("$vidfoldername/$(lpad(i, 4, '0')).png")
    end
    close(file)
    rm("$vidfoldername")
    close(w)
    fig
end

makefluxvid (generic function with 2 methods)

In [4]:
makefluxvid("cooling2", 3000)

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/ryhNR/src/JLD2.jl:287


LoadError: SystemError: opening file "../scratch/filament-instability/cooling2/down_front_mean.jld2": No such file or directory

In [ ]:
runname = "default"
@inline function getfluxplot(runname)
    foldername = "../scratch/filament-instability/$runname"
    filename = "down_front_mean.jld2"
    fluxfilename = "turbulent_flux.jld2"
    paramfilename = "parameters.jld2"
    frames, grid = jldopen("$foldername/$filename") do file
        keys(file["timeseries/t"]), file["serialized/grid"]
        end;
    xᶜᵃᵃ = xnodes(Center, grid)
    xᶠᵃᵃ = xnodes(Face, grid)
    zᵃᵃᶜ = znodes(Center, grid)
    zᵃᵃᶠ = znodes(Face, grid)

    file = jldopen("$foldername/$filename")
    qRifile = jldopen("$foldername/$qRifilename")
    fluxfile = jldopen("$foldername/$fluxfilename")
    
    init_time = jldopen("$foldername/$paramfilename") do file
        file["simulation_parameters"].init_time
    
    n = Observable(101)

    frame = @lift frames[$n]


    ts = [file["timeseries/t/$f"] for f in frames] .- file["timeseries/t/$f"]

    u = @lift file["timeseries/u_dfm/$($frame)"][:, 1, :]
    w = @lift file["timeseries/w_dfm/$($frame)"][:, 1, :]

    b = @lift file["timeseries/b_dfm/$($frame)"][:, 1, :]
    
    
    # Get the secondary cirulation streamfunction
    σ=3
    ψ = @lift imfilter(ψᶜᶜᶜ($u, $w, xᶜᵃᵃ, xᶠᵃᵃ, zᵃᵃᶜ, zᵃᵃᶠ), gaussian((σ, 0), (4σ+1, 5)), "circular")
    
    uflux = @lift fluxfile["timeseries/uFLUX/$($frame)"][:, 1, :]
    vflux = @lift fluxfile["timeseries/vFLUX/$($frame)"][:, 1, :]
    wflux = @lift fluxfile["timeseries/wFLUX/$($frame)"][:, 1, :]
    bflux = @lift fluxfile["timeseries/bFLUX/$($frame)"][:, 1, :]
    q = @lift imfilter(qRifile["timeseries/q/$($frame)"][:, 1, :], gaussian((σ, 0), (4σ+1, 5)), "circular")
    q_check = @lift ifelse.($q .< 0, 1, NaN)
    invRi = @lift imfilter(qRifile["timeseries/invRi/$($frame)"][:, 1, :], gaussian((σ, 0), (4σ+1, 5)), "circular")
    invRi_check = @lift ifelse.(1/0.95 .< $invRi .< 4, 1, NaN)
    title = @lift "t = $(round(ts[$n]; digits=2))"
    
    axis_kwargs = (; xlabel="x", ylabel="z", title, limits=(-2, 0, -0.12, 0))

    fig = Figure(resolution=(1200, 800))
    fig[1, 1:4] = Label(fig, title, fontsize=20)
    axu = Axis(fig[2, 1]; title=L"\langle w'u'\rangle", axis_kwargs...)
    axv = Axis(fig[2, 3]; title=L"\langle w'v'\rangle", axis_kwargs...)
    axw = Axis(fig[3, 1]; title=L"\langle w'w'\rangle", axis_kwargs...)
    axb = Axis(fig[3, 3]; title=L"\langle w'b'\rangle", axis_kwargs...)
    
    limuv = 1e-2
    limw = 1e-2
    limb = 1e-2
    htu = heatmap!(axu, xᶜᵃᵃ, zᵃᵃᶠ, uflux; colormap=:balance, colorrange=(-limuv, limuv))
    htv = heatmap!(axv, xᶜᵃᵃ, zᵃᵃᶠ, vflux; colormap=:balance, colorrange=(-limuv, limuv))
    htw = heatmap!(axw, xᶜᵃᵃ, zᵃᵃᶠ, wflux; colormap=:balance, colorrange=(-limw, limw))
    htb = heatmap!(axb, xᶜᵃᵃ, zᵃᵃᶠ, bflux; colormap=:balance, colorrange=(-limb, limb))
    
    for ax in [axu, axv, axw, axb]
        contour!(ax, xᶜᵃᵃ, zᵃᵃᶜ, ψ; colormap=:BrBG_10, levels=range(-0.012, 0.012, 40), alpha=1, linewidth=1)
        contour!(ax, xᶜᵃᵃ, zᵃᵃᶜ, b; color=(:black, 1), levels=range(-600, 200, 160), linewidth=1)
        #contour!(ax, xᶜᵃᵃ, zᵃᵃᶠ, invRi; color=:blue, levels=[1/0.95], alpha=0.5)
        #contour!(ax, xᶜᵃᵃ, zᵃᵃᶠ, invRi; color=:blue, levels=[1/0.25], linestyle=:dash, alpha=0.5)
        #contour!(ax, xᶜᵃᵃ, zᵃᵃᶜ, q; color=:red, levels=[0], alpha=0.5)
        #heatmap!(ax, xᶜᵃᵃ, zᵃᵃᶜ, q_check; colormap=[RGBAf(c.r, c.g, c.b, 0.1) for c in to_colormap(:reds)])
        #heatmap!(ax, xᶜᵃᵃ, zᵃᵃᶠ, invRi_check; colormap=[RGBAf(c.r, c.g, c.b, 0.1) for c in to_colormap(:blues)])
    end
    Colorbar(fig[2, 2], htu, label=L"\langle w'u'\rangle")
    Colorbar(fig[2, 4], htv, label=L"\langle w'v'\rangle")
    Colorbar(fig[3, 2], htw, label=L"\langle w'w'\rangle")
    Colorbar(fig[3, 4], htb, label=L"\langle w'b'\rangle")
    
    if i != nothing
        n[] = i
        return fig
    end
    
    vidfoldername = "output/videos/$runname/shearproduction"
    !ispath(vidfoldername) && mkpath(vidfoldername)
    w = ZipFile.Writer("$vidfoldername.zip");
    for i in 101:length(frames)
        n[] = i
        zipfile = ZipFile.addfile(w, "$(lpad(i, 4, '0')).png");
        save("$vidfoldername/$(lpad(i, 4, '0')).png", fig; resolution=(1200, 800))
        open(r -> write(zipfile, r), "$vidfoldername/$(lpad(i, 4, '0')).png")
        close(zipfile)
        rm("$vidfoldername/$(lpad(i, 4, '0')).png")
    end
    close(file)
    rm("$vidfoldername")
    close(w)
    fig
end